In [1]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 37.3 MB/s eta 0:00:00:00:0100:01


In [2]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 63.9 MB/s eta 0:00:00:00:0100:01


In [3]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder
import optuna
import lightgbm as lgb
import numpy as np

In [4]:
import duckdb
import pandas as pd

train_path = '/kaggle/input/leash-predict-chemical-bindings/train.parquet'
test_path = '/kaggle/input/leash-predict-chemical-bindings/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 35000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 35000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,98216988,O=C(N[C@@H](Cc1ccc(F)cc1)C(=O)O)OCC1c2ccccc2-c...,CNC(=O)c1ccc(N)cc1F,Cl.NCCC1CN(c2ncnc3[nH]ncc23)c2ccccc21,CNC(=O)c1ccc(Nc2nc(NCCC3CN(c4ncnc5[nH]ncc45)c4...,BRD4,0
1,278543698,O=C(O)Cc1ccc(NC(=O)OCC2c3ccccc3-c3ccccc32)cc1,NCC(O)COc1ccccc1Br,N#Cc1ncc(N)cc1C(F)(F)F,N#Cc1ncc(Nc2nc(NCC(O)COc3ccccc3Br)nc(Nc3ccc(CC...,HSA,0
2,145368917,O=C(Nc1c(Cl)cc(Cl)nc1C(=O)O)OCC1c2ccccc2-c2ccc...,Cl.Cl.NCCC(=O)Nc1ccncc1,Cc1ccc(C)c(CN)n1,Cc1ccc(C)c(CNc2nc(NCCC(=O)Nc3ccncc3)nc(Nc3c(Cl...,sEH,0
3,163978090,O=C(Nc1cc(C(=O)O)ccc1Br)OCC1c2ccccc2-c2ccccc21,Cl.NCC1CC(C(N)=O)=NO1,COc1cc(C)c(N)cn1,COc1cc(C)c(Nc2nc(NCC3CC(C(N)=O)=NO3)nc(Nc3cc(C...,HSA,0
4,166294371,O=C(Nc1cc(C(=O)O)ccc1C(F)(F)F)OCC1c2ccccc2-c2c...,Cn1nccc1CN1C[C@@H](F)C[C@H]1CN,Cc1cc(Cl)nnc1N,Cc1cc(Cl)nnc1Nc1nc(NC[C@@H]2C[C@H](F)CN2Cc2ccn...,BRD4,0


In [6]:
# ECFP 생성 함수
def generate_ecfp(molecule, radius=2, bits=2048):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

# 분자의 물리화학적 특성을 추출하는 함수
def generate_physchem_features(molecule):
    if molecule is None:
        return [np.nan] * 12
    return [
        Descriptors.MolWt(molecule),                   # 분자량
        Descriptors.MolLogP(molecule),                 # 로그 P
        Descriptors.NumHDonors(molecule),              # 수소 결합 공여자 수
        Descriptors.NumHAcceptors(molecule),           # 수소 결합 수용체 수
        Descriptors.TPSA(molecule)                     # TPSA (극성 표면적)
    ]

In [7]:
# 데이터 전처리
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)
df['physchem'] = df['molecule'].apply(generate_physchem_features)
df['ecfp'] = df['molecule'].apply(generate_ecfp)

In [8]:
# ECFP와 물리화학적 특성을 결합
df['features'] = df.apply(lambda row: row['ecfp'] + row['physchem'], axis=1)

In [9]:
# 단백질 이름을 원-핫 인코딩
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

## Train Model

In [10]:
# 최종 입력 데이터 생성
X = [features + list(protein) for features, protein in zip(df['features'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

In [11]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Optuna를 사용하여 하이퍼파라미터 최적화
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'num_leaves': trial.suggest_int('num_leaves', 31, 255),

    }

    model = lgb.LGBMClassifier(**param)
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    map_score = average_precision_score(y_test, y_pred_proba)
    return map_score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-05-31 06:38:37,391] A new study created in memory with name: no-name-4031ef84-b5ef-4503-8a02-d6d1c33c7d31
/tmp/ipykernel_33/2297067297.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
/tmp/ipykernel_33/2297067297.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
/tmp/ipykernel_33/2297067297.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6,

[LightGBM] [Info] Number of positive: 27969, number of negative: 28031
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.519117 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4493
[LightGBM] [Info] Number of data points in the train set: 56000, number of used features: 1858
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499446 -> initscore=-0.002214
[LightGBM] [Info] Start training from score -0.002214
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

[I 2024-05-31 06:39:05,363] Trial 0 finished with value: 0.9652201206769182 and parameters: {'n_estimators': 99, 'max_depth': 8, 'learning_rate': 0.19171048600266727, 'subsample': 0.9835131250866096, 'colsample_bytree': 0.9691701393277286, 'num_leaves': 163}. Best is trial 0 with value: 0.9652201206769182.
/tmp/ipykernel_33/2297067297.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
/tmp/ipykernel_33/2297067297.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
/tmp/ipykernel_33/2297067297.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0.

[LightGBM] [Info] Number of positive: 27969, number of negative: 28031
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.931588 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4493
[LightGBM] [Info] Number of data points in the train set: 56000, number of used features: 1858
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499446 -> initscore=-0.002214
[LightGBM] [Info] Start training from score -0.002214


[I 2024-05-31 06:39:41,900] Trial 1 finished with value: 0.9638720843661411 and parameters: {'n_estimators': 188, 'max_depth': 8, 'learning_rate': 0.06703000994099725, 'subsample': 0.9985354371708882, 'colsample_bytree': 0.9630258395923298, 'num_leaves': 39}. Best is trial 0 with value: 0.9652201206769182.
/tmp/ipykernel_33/2297067297.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
/tmp/ipykernel_33/2297067297.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
/tmp/ipykernel_33/2297067297.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0.

...

[I 2024-05-31 07:06:20,852] Trial 47 finished with value: 0.9714564123653937 and parameters: {'n_estimators': 262, 'max_depth': 9, 'learning_rate': 0.16358492240871536, 'subsample': 0.6150235210451214, 'colsample_bytree': 0.7734409216876837, 'num_leaves': 85}. Best is trial 32 with value: 0.9715741869326938.
/tmp/ipykernel_33/2297067297.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
/tmp/ipykernel_33/2297067297.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
/tmp/ipykernel_33/2297067297.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.

[LightGBM] [Info] Number of positive: 27969, number of negative: 28031
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.999262 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4493
[LightGBM] [Info] Number of data points in the train set: 56000, number of used features: 1858
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499446 -> initscore=-0.002214
[LightGBM] [Info] Start training from score -0.002214


[I 2024-05-31 07:06:56,257] Trial 48 finished with value: 0.966933237622045 and parameters: {'n_estimators': 267, 'max_depth': 10, 'learning_rate': 0.06708746924261558, 'subsample': 0.834225272951177, 'colsample_bytree': 0.7227821701882273, 'num_leaves': 32}. Best is trial 32 with value: 0.9715741869326938.
/tmp/ipykernel_33/2297067297.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
/tmp/ipykernel_33/2297067297.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
/tmp/ipykernel_33/2297067297.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0

[LightGBM] [Info] Number of positive: 27969, number of negative: 28031
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.006526 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4493
[LightGBM] [Info] Number of data points in the train set: 56000, number of used features: 1858
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499446 -> initscore=-0.002214
[LightGBM] [Info] Start training from score -0.002214
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

[I 2024-05-31 07:07:30,609] Trial 49 finished with value: 0.9701414178570871 and parameters: {'n_estimators': 250, 'max_depth': 8, 'learning_rate': 0.1298417955682621, 'subsample': 0.6674185913849512, 'colsample_bytree': 0.7464001060618694, 'num_leaves': 57}. Best is trial 32 with value: 0.9715741869326938.


In [13]:
# 최적의 하이퍼파라미터로 모델 생성
best_params = study.best_params
best_model = lgb.LGBMClassifier(**best_params)
best_model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 27969, number of negative: 28031
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.990723 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4493
[LightGBM] [Info] Number of data points in the train set: 56000, number of used features: 1858
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499446 -> initscore=-0.002214
[LightGBM] [Info] Start training from score -0.002214
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

LGBMClassifier(colsample_bytree=0.8127032950711086,
               learning_rate=0.20700096452492145, max_depth=9, n_estimators=238,
               num_leaves=105, subsample=0.6381176369635262)

## Submission

In [18]:
# 테스트 데이터 예측 및 저장
test_file = '/kaggle/input/leash-predict-chemical-bindings/test.csv'  
output_file = '/kaggle/working/submission3.csv'  # 출력 파일 경로

In [19]:
import os

In [20]:
for df_test in pd.read_csv(test_file, chunksize=100000):
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)
    df_test['physchem'] = df_test['molecule'].apply(generate_physchem_features)
    df_test['features'] = df_test.apply(lambda row: row['ecfp'] + row['physchem'], axis=1)
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))
    X_test = [features + list(protein) for features, protein in zip(df_test['features'].tolist(), protein_onehot.tolist())]
    probabilities = best_model.predict_proba(X_test)[:, 1]
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))